In [1]:
import os
import json
from hydra import compose, initialize
from omegaconf import OmegaConf, DictConfig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ml4cc.tools.evaluation import general as g
from ml4cc.tools.visualization import losses as l
from ml4cc.tools.visualization import regression as rv
from ml4cc.tools.visualization import classification as cv

with initialize(version_base=None, config_path="../config", job_name="test_app"):
    cfg = compose(config_name="main")

In [2]:
CLASSIFICATION_TRAININGS = {
    "two_step_minimal": "/home/laurits/ml4cc/training-outputs/20250622_FCC_twostep_minimal/two_step_minimal/",
    "two_step_pf_masked": "/home/laurits/ml4cc/training-outputs/20250620_FCC_twostep_masked/two_step_pf/",
    "two_step_pf": "/home/laurits/ml4cc/training-outputs/20250618_FCC_twostep/two_step_pf/"
}

REGRESSION_TRAININGS = {
    "one_step": "/home/laurits/ml4cc/training-outputs/20250622_FCC_onestep/one_step/",
    "two_step_minimal": "/home/laurits/ml4cc/training-outputs/20250622_FCC_twostep_minimal/two_step_minimal/",
    # Should add here also the clusterization trainings
}
RESULTS_DIR = "/home/laurits/tmp/results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Classification

In [3]:
# Loss evaluation

loss_results = {}
lp = l.LossesMultiPlot(loss_name="BCE", plot_train_losses=True, x_max=-1)
for algo, training_dir in CLASSIFICATION_TRAININGS.items():
    metrics_path = os.path.join(training_dir, "logs/metrics.csv")
    losses = g.filter_losses(metrics_path=metrics_path)
    losses_output_path = os.path.join(RESULTS_DIR, "BCE_losses.png")
    loss_results[algo] = losses
lp.plot_algorithms(results=loss_results, output_path=losses_output_path)


In [4]:
# Comparing ROC, AUC, etc.

all_results = {}
loss_results = {}
for algo, training_dir in CLASSIFICATION_TRAININGS.items():
    results_path = os.path.join(training_dir, "results/results.json")
    metrics_path = os.path.join(training_dir, "logs/metrics.csv")
    all_results[algo] = {}
    best_loss = np.min(g.filter_losses(metrics_path)["val_loss"])
    loss_results[algo] = {"best_losses": [best_loss]}
    with open(results_path, 'rt') as in_file:
        results = json.load(in_file)
        for pid in results.keys():
            all_results[algo][pid] = results[pid]["global"]



lsp = l.LossesStackPlot(loss_name="BCE")
lsp_output_path = os.path.join(RESULTS_DIR, "cls_loss_stack.png")
lsp.plot_algorithms(loss_results, lsp_output_path)

asp = cv.AUCStackPlot()
asp_output_path = os.path.join(RESULTS_DIR, "AUC_stack.png")
asp.plot_algorithms(all_results, asp_output_path)

two_step_minimal
dict_keys(['resolution', 'median', 'AUC'])
two_step_pf_masked
dict_keys(['FPRs', 'TPRs', 'energies', 'AUC_mean', 'AUC', 'ROC'])
two_step_pf
dict_keys(['FPRs', 'TPRs', 'energies', 'AUC_mean', 'AUC', 'ROC'])


# Regression

In [5]:
# Loss evaluation

loss_results = {}
lp = l.LossesMultiPlot(loss_name="MSE", plot_train_losses=True)
for algo, training_dir in REGRESSION_TRAININGS.items():
    metrics_path = os.path.join(training_dir, "logs/metrics.csv")
    losses = g.filter_losses(metrics_path=metrics_path)
    losses_output_path = os.path.join(RESULTS_DIR, "MSE_losses.png")
    loss_results[algo] = losses
lp.plot_algorithms(results=loss_results, output_path=losses_output_path)

In [7]:
# Comparing the resolution
all_results = {}
loss_results = {}
for algo, training_dir in REGRESSION_TRAININGS.items():
    results_path = os.path.join(training_dir, "results/results.json")
    metrics_path = os.path.join(training_dir, "logs/metrics.csv")
    all_results[algo] = {}
    if not algo == "two_step_minimal":
        best_loss = np.min(g.filter_losses(metrics_path)["val_loss"])
        loss_results[algo] = {"best_losses": [best_loss]}
    with open(results_path, 'rt') as in_file:
        results = json.load(in_file)
        for pid in results.keys():
            all_results[algo][pid] = results[pid]["global"]

rsp = rv.RegressionStackPlot()
rsp_output_path = os.path.join(RESULTS_DIR, "resolution_stack.png")
rsp.plot_algorithms(all_results, rsp_output_path)

lsp = l.LossesStackPlot(loss_name="MSE")
lsp_output_path = os.path.join(RESULTS_DIR, "regression_loss_stack.png")
lsp.plot_algorithms(loss_results, lsp_output_path)

dict_keys(['2', '4', '6', '8', '10', '165', '180', 'global'])
dict_keys(['global'])
